On possède un dataset de tweets, et on cherche à faire du clustering pour identifier les tweets parlant de sujets similaires.

Pour ce TP, je ne vous ai passé qu'une partie du dataset. Si votre solution marche bien, ou que vous avez besoin de plus de données, et que vous avez un meilleur pc que moi,
vous pouvez récupérer le reste du dataset ici :
    
https://github.com/fivethirtyeight/russian-troll-tweets


In [20]:
import os
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt

dataset_file_1 = Path("IRAhandle_tweets_1.txt")
df = pd.read_csv(dataset_file_1)

In [21]:
tweets_list = df["content"].values

In [22]:
tweets_list

array(['"We have a sitting Democrat US Senator on trial for corruption and you\'ve barely heard a peep from the mainstream media." ~ @nedryun https://t.co/gh6g0D1oiC',
       'Marshawn Lynch arrives to game in anti-Trump shirt. Judging by his sagging pants the shirt should say Lynch vs. belt https://t.co/mLH1i30LZZ',
       'Daughter of fallen Navy Sailor delivers powerful monologue on anthem protests, burns her NFL packers gear.  #BoycottNFL https://t.co/qDlFBGMeag',
       ...,
       'How we can rebuild trust in a UK divided by inequality and suspicion https://t.co/cz5y1r9ikh',
       'John Humphrys accused of patronising Angela Rayner on Today programme https://t.co/1nWj11h0tR',
       "Fossilized poop found in 180-million-year-old 'ghost-gut' reveals a dinosaur's last meal https://t.co/BF6bsG3z39"],
      dtype=object)

In [23]:
from gensim.models import KeyedVectors
# Si vous n'avez pas déjà un word2vec entrainé : https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM/edit?usp=sharing

filename = 'GoogleNews-vectors-negative300.bin'
model = KeyedVectors.load_word2vec_format(filename, binary=True)
model.fill_norms()

# Comment utiliser une représentation des mots pour faire du clustering de phrase ?

- Solution 1 : obtenir la representation d'un mot, puis faire votre propre representation du tweet à partir de celle la

In [24]:
v1 = model.get_vector("Bordeaux")
print("V1_infer", v1)

V1_infer [-4.29687500e-02 -2.12402344e-02 -3.07617188e-02  2.15820312e-01
  4.14062500e-01  1.89453125e-01 -1.41601562e-01 -2.07031250e-01
  2.94189453e-02  2.34375000e-01  3.03955078e-02  1.34765625e-01
  2.24609375e-01 -2.57812500e-01 -1.15234375e-01  3.12500000e-01
 -9.03320312e-02  1.87500000e-01  2.69775391e-02  2.79296875e-01
  1.80664062e-02 -2.09960938e-01  5.63964844e-02 -3.29589844e-02
 -2.82287598e-03  3.41796875e-01  1.59912109e-02 -1.20605469e-01
 -7.37304688e-02  1.33789062e-01 -5.63964844e-02  2.25585938e-01
 -2.81250000e-01 -9.17968750e-02 -1.34765625e-01  8.25195312e-02
 -4.37011719e-02 -1.90429688e-01 -1.30859375e-01 -4.47265625e-01
 -2.07031250e-01  1.50390625e-01  5.63964844e-02  1.95312500e-01
  2.45117188e-01  3.95507812e-02  3.37890625e-01  5.63964844e-02
 -2.51953125e-01  9.46044922e-03 -4.17968750e-01  9.76562500e-02
 -4.78515625e-01 -5.83496094e-02 -5.58593750e-01  2.94921875e-01
 -3.71093750e-01  3.14941406e-02 -6.64062500e-02 -2.27539062e-01
 -2.98828125e-01

- Solution 2 : utiliser la méthode fournie pour comparer directement deux ensembles de mots

In [25]:
model.n_similarity(["I", "tweet"],["I","use","twitter"])

0.8098895

In [26]:
model.n_similarity(["I", "tweet"],["I","use","facebook"])

0.69237375

- Solution 3 : doc2vec ou autre embedding de phrases/documents plus élaborés

### A partir du jeu de donnée, effectuer un clustering des tweets

Nettoyez votre jeu de données rapidement :

    - ne gardez que 1000 phrase pour l'instant (surtout si votre pc est lent !)
    - tokenizez chaque phrase
    - enlevez les stops words et les mots non connus par votre model pré-entrainé (non présents dans valid_tokens ci dessous)

In [27]:
from nltk import word_tokenize
from nltk.corpus import stopwords  
  
valid_tokens = set(model.key_to_index.keys())
stop_words = set(stopwords.words('english'))
nb_sentence = 1_000

In [33]:
def tokenize(tweet: str) -> str:
    tokens = word_tokenize(tweet)
    tokens = [tk for tk in tokens if tk not in stop_words and tk.isalpha()]
    return str(
       )

100%|██████████| 1/1 [00:00<00:00, 1998.24it/s]


['``',
 'We',
 'sitting',
 'Democrat',
 'US',
 'Senator',
 'trial',
 'corruption',
 "'ve",
 'barely',
 'heard',
 'peep',
 'mainstream',
 'media',
 '.',
 "''",
 '~',
 '@',
 'nedryun',
 'https',
 ':',
 '//t.co/gh6g0D1oiC']

In [34]:
valid_nd_stop =  [tk for tk in valid_tokens if tk in valid_tokens]
tokenized_sentences = tweets_list[:nb_sentence].apply(lambda x: tokenize(x))

AttributeError: 'numpy.ndarray' object has no attribute 'apply'

Créez une matrice de similarité :

    - elle est de taille (nb_sentence, nb_sentence), ici (1000,1000)
    - la case i,j indique la distance entre la phrase i et la phrase j, calculée au moyen de model.n_similarity
    - pour l'algorithme de spectral clustering, elle doit valoir 1 si les phrases sont identiques, et 0 si les phrases sont completements différentes (ou une des phrase est vide)

In [ ]:
similarity_matrix = np.zeros((nb_sentence, nb_sentence))

# TODO : remplir la matrice

### Entrainons notre algorithme de clustering

- comme souvent en ML, une fois que l'on a fait le preprocessing adéquat, on a fait 80% du job !

In [ ]:
from sklearn.cluster import SpectralClustering
import numpy as np

cls = SpectralClustering( affinity="precomputed")
preds = cls.fit_predict(similarity_matrix)

### Visualisons tout ça

Avec une pca ?

In [ ]:
data_as_array = np.zeros((nb_sentence, 68))
for i, sen in enumerate(tokenized_sentences_int[:1000]):
    data_as_array[i, :len(sen)] = sen

In [ ]:
from sklearn.decomposition import PCA

tokenized_sentences_int 
transformed = PCA(n_components=2).fit_transform(data_as_array)

In [ ]:
plt.scatter(x=transformed[:,0], y=transformed[:,1], c=preds)

Regardons le contenu:

In [ ]:
for i in np.where(preds == 3)[0]:
    print(tokenized_sentences[i])

 Le cluster à l'air de parler de Media et politique ?
 
 Comment améliorer :
- plus de cluster
- plus que 1000 exemples en train
- semi-supervisés avec phrases d'exemples pour chaque cluster
- meilleure métrique de distance (doc2vec, transformers, ...)


Question bonus : Pourquoi à votre avis je n'ai pas voulu utiliser l'algorithme de clustering DBSCAN ici ?
- vous pouvez vérifier votre intuition en inversant la matrice de distance et en la fournissant à dbscan !